In [ ]:
import sys, os
sys.path.append('..')
import avstack
import avapi
import numpy as np
import quaternion
%load_ext autoreload
%autoreload 2

obj_data_dir_k = '/data/spencer/KITTI/object'
raw_data_dir_k = '/data/spencer/KITTI/raw'
data_dir_n = '/data/spencer/nuScenes/'

# -- static dataset
KOD = avapi.kitti.KittiObjectDataset
KDM = KOD(obj_data_dir_k, 'training')

# -- raw dataset
KRD = avapi.kitti.KittiRawDataset(raw_data_dir_k)

In [ ]:
idx_date = 0
idx_seq = 2
exp_path = KRD.convert_sequence(KRD.get_available_dates()[idx_date], idx_seq=idx_seq,
                                max_frames=None, max_time=None, tracklets_req=True)
KDM = KOD(obj_data_dir_k, exp_path)

## Test visuals

In [ ]:
frame = KDM.frames[30]
img = KDM.get_image(frame, sensor='image-2')
pc = KDM.get_lidar(frame)
objects = KDM.get_objects(frame)
avapi.visualize.show_objects_on_image(img, objects, projection='3d', inline=True)
avapi.visualize.show_lidar_on_image(pc, img)

## Test Detection

In [ ]:
M2D = avstack.modules.perception.object2dfv.MMDetObjectDetector2D(
    model='cascade_mask_rcnn',dataset='kitti', gpu=1)

M = avstack.modules.perception.object3d.MMDetObjectDetector3D(
    model='pgd',dataset='kitti', gpu=1)

In [ ]:
frame = KDM.frames[150]
img = KDM.get_image(frame, sensor='image-2')
pc = KDM.get_lidar(frame)
objects = KDM.get_objects(frame)

# outputs = M2D(frame, img, 'test')
outputs = M(frame, img, 'test')
# outputs = M(frame, pc, 'test')
avapi.visualize.show_image_with_boxes(img, outputs.data, inline=True)

In [ ]:
outputs.data

## Test Raw Dataset Scenes

In [ ]:
KRD_ = avapi.kitti.KittiRawDataset
RD = KRD_(raw_data_dir_k)

In [ ]:
def make_krdm(idx_date, idx_seq, remake_data=False):
    dates = RD.get_available_dates()
    sequence_ids = RD.get_sequence_ids_at_date(dates[idx_date], True)
    if remake_data:
        exp_path = RD.convert_sequence(dates[idx_date], idx_seq=idx_seq,
                    max_frames=None, max_time=None, tracklets_req=True,
                    path_append='-experiment', verbose=True)
    else:
        _, exp_path, _ = RD.get_converted_exp_path(dates[idx_date], idx_seq=idx_seq,
                        tracklets_req=True, path_append='-experiment')
        if not os.path.exists(exp_path):
            raise RuntimeError('No data found at %s' % exp_path)
    RDM = KOD(obj_data_dir_k, exp_path.split('/')[-1])
    return RDM

In [ ]:
available_scenes = []

for idx_date in range(len(RD.get_available_dates())):
    idx_seq = 0
    while True:
        try:
            DM = make_krdm(idx_date, idx_seq, remake_data=False)
        except RuntimeError as e:
            DM = make_krdm(idx_date, idx_seq, remake_data=True)
        except IndexError as e:
            print(f'Combo ({idx_date}, {idx_seq}) does not work')
            break
        print(f'Found scene combo ({idx_date}, {idx_seq})')
        available_scenes.append((idx_date, idx_seq))
        idx_seq += 1
